In [3]:
import pandas as pd

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [4]:
yelpparsed=pd.read_csv('../data/yelpdata.csv')
foursquareparsed=pd.read_csv('../data/foursqdata.csv')
citybikedata=pd.read_csv('../data/data.csv')

In [9]:
citybikedata.rename(columns={'name': 'station'}, inplace=True)

In [22]:
mergeddf=pd.merge(citybikedata,yelpparsed,on=['station'],how='inner',suffixes=('city','yelp'))
finaldf=pd.merge(mergeddf,foursquareparsed,on=['station'],how='inner',suffixes=('','foursq'))
finaldf=finaldf.rename(columns={'distance': 'distanceyelp','name':'poiname', 'rating':'ratingyelp','popularity':'popularityyelp'})

In [23]:
finaldf.head()

,station,latitude,longitude,free_bikes,empty_slots,total_bike_cap,distanceyelp,poiname,location.address,ratingyelp,distancefoursq,namefoursq,location.addressfoursq,ratingfoursq,popularityyelp,categories
0,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,773,Union Market,2306 Jefferson Ave,9.4,0.984966,bar
1,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,912,Liberty Public House,418 N 25th St,8.7,0.975994,bar
2,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,604,Kahlo’s Taqueria and Bar,718 N 23rd St,7.8,0.971639,bar
3,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,917,Leisure Business Advisors,2010 Princess Anne Ave,NaN,0.283975,office
4,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,4.2,999,Armstrong Renaissance,1665 N 31st St,NaN,0.324007,office


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [24]:
finaldf['ratingyelp']=finaldf[['ratingyelp']]*2
finaldf.head()

,station,latitude,longitude,free_bikes,empty_slots,total_bike_cap,distanceyelp,poiname,location.address,ratingyelp,distancefoursq,namefoursq,location.addressfoursq,ratingfoursq,popularityyelp,categories
0,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,773,Union Market,2306 Jefferson Ave,9.4,0.984966,bar
1,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,912,Liberty Public House,418 N 25th St,8.7,0.975994,bar
2,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,604,Kahlo’s Taqueria and Bar,718 N 23rd St,7.8,0.971639,bar
3,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,917,Leisure Business Advisors,2010 Princess Anne Ave,NaN,0.283975,office
4,East End Library,37.539779,-77.41146,0,6,6,645.388529,The Roosevelt,623 N 25th St,8.4,999,Armstrong Renaissance,1665 N 31st St,NaN,0.324007,office


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [27]:
import sqlite3
from sqlite3 import Error as e

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection successful")
    except Error as e:
        print(f'The error {e}')
    return connection



connection = create_connection('../data/db')

Connection successful


creating function to execute queries

In [28]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as e:
        print(f"The error '{e}' occurred")

Creating and filling in stations table

In [31]:
create_stations_table = """
CREATE TABLE IF NOT EXISTS stations (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  latitude INTEGER,
  longitude INTEGER,
  free_bikes INTEGER,
  empty_slots INTEGER,
  total_bike_cap INTEGER
); """

execute_query(connection, create_stations_table)  


In [32]:
cursor = connection.cursor()
for index, row in citybikedata.iterrows():
    create_station = f"""
    INSERT INTO 
        stations(name, latitude, longitude, free_bikes, empty_slots, total_bike_cap)
    VALUES
        ('{row['station']}', {row['latitude']}, {row['longitude']},{row['free_bikes']}, {row['empty_slots']}, {row['total_bike_cap']})
    """
    cursor.execute(create_station)
connection.commit()

Query successful


Creating and inserting yelp table

creating read statement function

In [39]:
check_stations_table = """
    SELECT *
    FROM stations
    LIMIT 10

"""



Look at the data before and after the join to validate your data.

In [40]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [41]:
stations = execute_read_query(connection, check_stations_table)

for station in stations:
    print(station)

(1, 'East End Library', 37.539778925345864, -77.41146002560403, 0, 6, 6)
(2, 'Sydney Park', 37.54696765790552, -77.45659857991984, 0, 16, 16)
(3, 'Broad & Harrison', 37.55147166118693, -77.45193690061576, 0, 19, 19)
(4, 'Broad & Lombardy', 37.55371555769088, -77.45568104693, 0, 23, 23)
(5, 'Broad Rock Library', 37.48329106243563, -77.47938997776974, 0, 10, 10)
(6, 'Abner Clay Park', 37.54887732047105, -77.44269669055932, 0, 16, 16)
(7, 'Center Stage', 37.541363013108416, -77.43708378535466, 0, 19, 19)
(8, 'Main Street Station', 37.53458546847389, -77.43012148479552, 0, 16, 16)
(9, 'Carter Jones Park', 37.51493677847786, -77.46047501956636, 0, 16, 16)
(10, 'Kanawha Plaza', 37.53713573652868, -77.43928223848339, 0, 19, 19)
